In [84]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from pprint import pprint

In [90]:
_tags = pd.read_csv("./datasets/healthy-sql/t_comment.csv", usecols=range(2,4)).dropna()
tags = _tags.groupby("essay_id").agg(list)
essay_id_to_category = pd.read_csv('./datasets/healthy-sql/t_health_category.csv',usecols=range(0, 2)).dropna()
essay = pd.read_csv('./datasets/healthy-sql/t_essay.csv',usecols=range(0,3)).dropna()

essay = pd.merge(essay, essay_id_to_category, left_on='essay_type', right_on='id', how='left')
essay = essay[['id_x', 'title_x', 'title_y']].rename(columns={'id_x': 'essay_id', 'title_x': 'title', 'title_y': 'category'})

essayIndex = set(essay.index) & set(tags.index)
new_tags = tags.loc[list(essayIndex)]
ret = essay.join(new_tags)

essay_dataset = pd.DataFrame(
    map(
        lambda x: (x[1], x[2], x[3], x[4]+[x[3]]) if x[4] is not np.nan else (x[1], x[2], x[3], []),
        ret.itertuples()
    ),
    columns=["essayId", "title", "category", "comments"]
)
essay_dataset

,essayId,title,category,comments
0,1,你知道什么是营养学吗？,营养学入门,[]
1,2,了解营养学分类，助您健康生活,营养学入门,"[打卡, 好文章, 图文并茂, 好难, 营养学入门]"
2,3,营养素是人体的“燃料”，你摄入足够了吗？,营养学入门,"[顶一个, 简单, 营养学入门]"
3,4,不足与过量如何影响我们的身体？,营养学入门,"[厉害了, 厉害了, 营养学入门]"
4,5,至关重要的饮食计划,营养学入门,"[好难, 顶一个, 营养学入门]"
...,...,...,...,...
76,95,腿部训练的方法和技巧,HIIT训练,[]
77,96,胸部训练的方法和技巧,短程游泳训练,[]
78,97,背部训练的方法和技巧,单车骑行训练,[]
79,98,肩部训练的方法和技巧,爬山徒步训练,[]


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/ml-latest-small/t_comment.csv'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.pipeline import make_pipeline

def create_movie_profile(essay_dataset):
    '''
    分析提取topn关键词
    '''
    dataset = essay_dataset["comments"].values

    # 构建词袋模型
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform(dataset)

    # 计算TF-IDF值
    tfidf_transformer = TfidfVectorizer(norm=None)
    tfidf_matrix = tfidf_transformer.fit_transform(bow_matrix)

    # 标准化TF-IDF向量
    normalized_tfidf_matrix = normalize(tfidf_matrix, norm='l2', axis=1)

    # 获取关键词
    feature_names = vectorizer.get_feature_names()
    movie_profile = {}
    for i, mid in enumerate(essay_dataset.index):
        vector = normalized_tfidf_matrix[i]
        indices = vector.indices
        scores = vector.data
        top_n_indices = indices[np.argsort(scores)[-10:]]
        top_n_keywords = [feature_names[idx] for idx in top_n_indices]
        movie_profile[mid] = dict(zip(top_n_keywords, scores[top_n_indices]))

    return movie_profile

pprint(create_movie_profile(essay_dataset))


ValueError: empty vocabulary; perhaps the documents only contain stop words

,title,genres
movieId,,
1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,Grumpier Old Men (1995),"[Comedy, Romance]"
4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
5,Father of the Bride Part II (1995),[Comedy]
...,...,...
193581,Black Butler: Book of the Atlantic (2017),"[Action, Animation, Comedy, Fantasy]"
193583,No Game No Life: Zero (2017),"[Animation, Comedy, Fantasy]"
193585,Flint (2017),[Drama]
